In [155]:
import pandas as pd

In [156]:
fe = pd.read_csv('Fashione EAN 18-11-25.csv', dtype=str)
fe['Offer SKU'] = fe['PLU']
fe = fe[['Style', 'Colour', 'Size', 'Offer SKU']]
fe.head(2)

,Style,Colour,Size,Offer SKU
0,007,BLK/BLUE,S,0002875800
1,007,BLK/BLUE,M,0002875801


In [157]:
offers = pd.read_csv('offers (82).csv', delimiter=';', dtype=str)
offers['RRP'] = offers['Original price'].astype(float)
offers['Sale price'] = offers['Discount price'].astype(float)
offers['Stock'] = offers['Quantity']
offers = offers[['Offer SKU', 'Stock', 'RRP', 'Sale price']]
offers.head(2)

,Offer SKU,Stock,RRP,Sale price
0,0004892908,3,209.0,NaN
1,0004828001,2,199.0,NaN


In [158]:
df = pd.read_csv('transaction-logs (51).csv', delimiter=';', dtype=str)

df['Offer SKU'] = df['Offer SKU'].replace(['0004925700', '0004928100', '0004928200', '0004928800', '0004555700', '0004933600'], ['0005346800', '0005347000', '0005346900', '0005347100', '0005310300', '0005347200'])

df['Category Label'] = df['Category Label'].replace(['Womens Flats', 'Womens Boots', 'Womens Nightwear'], 'Womens Footwear')
df['Category Label'] = df['Category Label'].replace(['Mens Shoes'], 'Mens Footwear')
df['Category Label'] = df['Category Label'].replace(['Luggage'], 'Mens Bags & Wallets')
df['Category Label'] = df['Category Label'].replace(['Soft Furnishings', 'Decorative Accessories',
                                                     'Womens Hair Accessories', 'Childrens Accessories',
                                                     'Hot & Cold Packs', 'Womens Hats', 'Mens Hats',
                                                     'Womens Leggings', 'Stationery'], 'Other Accessories')
dict_category = pd.Series(df['Category Label'].values,index=df['Offer SKU']).to_dict()

df = df[df['Type'] == 'Order amount']
df.dropna(subset='Quantity', inplace=True)
df['Quantity'] = df['Quantity'].astype(int)
df['Amount'] = df['Amount'].astype(float)
temp = df['Date created'].str.split(' - ', expand=True)
df['Date created'] = pd.to_datetime(temp[0], format='%d/%m/%Y')

raw_df = pd.merge(df, fe, on='Offer SKU', how='left')
raw_df.to_csv('Raw file for graph.csv', index=False)

df['Year'] = df['Date created'].dt.year
df = df[df['Year'].isin([2023, 2024, 2025])]
df = df.groupby(['Offer SKU', 'Year'])[['Quantity']].sum().reset_index()
df

,Offer SKU,Year,Quantity
0,0000072400,2024,23
1,0000072400,2025,97
2,0000072500,2024,21
3,0000072500,2025,25
4,0000072600,2023,17
...,...,...,...
7115,S27756,2023,3
7116,S27757,2023,2
7117,S27758,2023,7
7118,S27759,2023,1


In [165]:
raw_df.columns

Index(['Date created', 'Date received', 'Transaction Date', 'Shop',
       'Order number', 'Invoice number', 'Transaction Number', 'Quantity',
       'Category Label', 'Offer SKU', 'Description', 'Type', 'Payment status',
       'Payment reference', 'Amount', 'Debit', 'Credit', 'Balance', 'Currency',
       'Customer order reference', 'Shop order reference',
       'Billing cycle date', 'Shop ID', 'Order line ID', 'Refund ID',
       'Sales channel', 'Style', 'Colour', 'Size'],
      dtype='object')

In [159]:
df_2023 = df[df['Year'] == 2023].copy()
df_2023.rename(columns={'Quantity': '2023'}, inplace=True)
df_2023 = df_2023[['Offer SKU', '2023']].copy()
df_2024 = df[df['Year'] == 2024]
df_2024.rename(columns={'Quantity': '2024'}, inplace=True)
df_2024 = df_2024[['Offer SKU', '2024']].copy()
df_2025 = df[df['Year'] == 2025]
df_2025.rename(columns={'Quantity': '2025'}, inplace=True)
df_2025 = df_2025[['Offer SKU', '2025']]
df = pd.merge(df_2025, df_2024, on='Offer SKU', how='outer')
df = pd.merge(df, df_2023, on='Offer SKU', how='outer')

df = pd.merge(df, fe, on='Offer SKU', how='left')
df = df.groupby(['Style', 'Colour', 'Offer SKU'])[['2023', '2024', '2025']].sum().reset_index()

df = pd.merge(df, offers, on='Offer SKU', how='left')
df.dropna(subset=['Style'], inplace=True)

C:\Users\retai\AppData\Local\Temp\ipykernel_17340\3526563117.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2024.rename(columns={'Quantity': '2024'}, inplace=True)
C:\Users\retai\AppData\Local\Temp\ipykernel_17340\3526563117.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2025.rename(columns={'Quantity': '2025'}, inplace=True)


In [160]:
df['2023'] = df['2023'].astype(int)
df['2024'] = df['2024'].astype(int)
df['2025'] = df['2025'].astype(int)
df['Stock'] = df['Stock'].fillna(0)
df['RRP'] = df['RRP'].fillna('-')
df['Sale price'] = df['Sale price'].fillna('-')

df['Category'] = df['Offer SKU']
df['Category'] = df['Category'].map(dict_category)

In [161]:
df.head()

,Style,Colour,Offer SKU,2023,2024,2025,Stock,RRP,Sale price,Category
0,09-192,PURPLE,0001991800,0,1,0,0,-,-,Womens Bags & Purses
1,10157,BLACK,0002080401,0,2,10,1,189.0,139.99,Womens Jackets & Coats
2,10157,BLACK,0002080402,0,2,18,0,189.0,139.99,Womens Jackets & Coats
3,10157,BLACK,0002080403,0,2,15,2,189.0,139.99,Womens Jackets & Coats
4,10157,BLACK,0002080404,0,2,15,2,189.0,139.99,Womens Jackets & Coats


In [162]:
df.to_csv('Data.csv', index=False)